In [ ]:
from loader.data_loader import REFIT_Loader

In [ ]:
refit = REFIT_Loader()

## Fetch available appliances for a selected house

In [ ]:
refit.get_appliance_names(house_number=3)

## Load data for selected appliance (selected houses)

In [ ]:
%%time
washing_machine_unsampled = refit.get_appliance_data('washingmachine', houses=[2,3])
washing_machine = refit.get_appliance_data('washingmachine', houses=[2,3])

In [ ]:
type(washing_machine)

## Resample data 

In [ ]:
%%time
washing_machine.resample(sampling_period='8s', fill_value=0.0, window_limit=3.0)

In [ ]:
washing_machine_unsampled.data[2]

In [ ]:
washing_machine.data[2]